Setting up the environment

Import modules and initialize

In [ ]:
import os
from dotenv import load_dotenv
import gradio as gr  # This defines 'gr'

# 1. Load the .env file BEFORE doing anything else
load_dotenv("../.env")

# 2. Check if the key is actually there (for debugging)
if not os.getenv("OPENAI_API_KEY"):
    print("❌ ERROR: OPENAI_API_KEY not found in .env!")
else:
    print("✅ API Key loaded successfully.")

# 3. NOW import your custom modules
from src.data_processor import process_input
from src.llm_processor import generate_script
from src.tts_generator import generate_audio

Setting Up Open AI API key access

In [ ]:
import os
from dotenv import load_dotenv

# The "../" tells Python to look one folder up (in the WEEK 1 root)
load_dotenv("../.env")

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ API Key successfully loaded!")
else:
    print("❌ API Key not found. Check your .env file location.")

Step 3: Text Pre-processing & Chunking

Ensure the text fits within API character limits.

In [ ]:
import re
import os

# Load text from Text input folder
rec_dir = 'Text input'
txts = [f for f in os.listdir(rec_dir) if f.lower().endswith('.txt')]
if not txts:
    raise FileNotFoundError(f"No .txt files found in {rec_dir!r}")

fname = os.path.join(rec_dir, txts[0])
with open(fname, 'r', encoding='utf-8') as f:
    full_text = f.read()

print(f"✅ Loaded {len(full_text)} characters from {fname}")

def chunk_text_by_sentences(text, max_chars=4000):
    """
    Splits text into chunks of max_chars, ensuring we don't 
    break sentences in the middle.
    """
    # 1. Basic cleaning: remove extra whitespace/newlines
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 2. Split text into sentences using regex
    sentences = re.split(r'(?<=[.!?]) +', text)
    
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chars:
            current_chunk += (sentence + " ")
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

text_chunks = chunk_text_by_sentences(full_text)

print(f"Processed {len(text_chunks)} chunks for TTS.")
for i, chunk in enumerate(text_chunks):
    print(f"Chunk {i+1} length: {len(chunk)} characters")

Define the logic wrapper

In [ ]:
def create_podcast(input_text, speaker_voice, file_input):
    print(f"🎙️ Process started for voice: {speaker_voice}")
    
    # Determine source
    content = process_input(input_text, file_input)
    
    if not content or content.strip() == "":
        return "⚠️ Error: No content found to process.", None
    
    try:
        print("--- Generating Script ---")
        script = generate_script(content)
        
        print("--- Generating Audio ---")
        audio_path = generate_audio(script, voice=speaker_voice)
        
        return script, audio_path
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return f"An error occurred: {str(e)}", None

Build the ui with blocks

3. How the Data Flows

Input: The user pastes text into the input_data box.

Trigger: The generate_btn click sends that text to create_podcast.

Processing: * llm_processor.py turns the text into a dialogue.

tts_generator.py saves an .mp3 file to a temporary directory.

Output: Gradio updates the script_out text box and the audio_out player simultaneously.

Adding templates

In [ ]:
import gradio as gr
import os
from dotenv import load_dotenv

# 1. KILL existing processes first
gr.close_all() 

load_dotenv()

# 2. RE-IMPORT to ensure the latest version of your .py files is used
import src.data_processor as dp
import src.llm_processor as lp
import src.tts_generator as tg
import importlib
importlib.reload(dp) # This forces the notebook to see your PDF fix
importlib.reload(lp)
importlib.reload(tg)

def create_podcast(input_text, speaker_voice, file_input):
    try:
        # Use the reloaded modules
        content = dp.process_input(input_text, file_input)
        if not content: return "No content found.", None
        
        script = lp.generate_script(content)
        audio_path = tg.generate_audio(script, voice=speaker_voice)
        
        return script, audio_path
    except Exception as e:
        return f"Error: {str(e)}", None

with gr.Blocks() as demo:
    gr.Markdown("# 🎙️ AI Podcast Studio")
    with gr.Row():
        with gr.Column():
            input_data = gr.Textbox(label="URL or Text")
            file_input = gr.File(label="Upload PDF", file_types=[".pdf"])
            voice_opt = gr.Dropdown(choices=["Alloy", "Echo", "Shimmer"], label="Voice", value="Alloy")
            generate_btn = gr.Button("Generate")
        with gr.Column():
            script_out = gr.Textbox(label="Script")
            audio_out = gr.Audio(label="Podcast")

    generate_btn.click(
        fn=create_podcast,
        inputs=[input_data, voice_opt, file_input],
        outputs=[script_out, audio_out]
    )

# Use inline=False to open in a fresh browser tab
demo.queue().launch(debug=True, inline=False)